In [20]:
import paramiko
from getpass import getpass

# Create Password Prompt

thePass = getpass(prompt="Enter your Password: ")

# Host Information

host = "192.168.6.71"
port = "2222"
username = "micah.kezar"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed.")

In [21]:
# Task 2
sftp = ssh.open_sftp()

# List of files to download
sftp.put('kraken','/home/micah.kezar/kraken')

command1 = 'chmod 777 ./kraken'
command2 = 'sudo -S ./kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin '

# Note to self, you can run as many commands as you want in ssh.exec_command if you seperate the commands with \n
stdin, stdout, stderr = ssh.exec_command(command1 + "\n" + command2)

stdin.write(password + "\n")

for line in iter(stdout.readline,""):
    print(line,end="")
    

INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0013] DETECTION! Malicious process detected as foundGoBinary  pid=519423 process=snapd
WARN[0014] DETECTION! Malicious process detected as foundGoBinary  pid=561559 process=egrep
INFO[0016] Scanning autoruns...                         
WARN[0016] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
WARN[0017] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd
INFO[0017] Scanning the filesystem (this can take several minutes)... 
ERRO[0017] Some malicious artifacts have been detected on this system: 
ERRO[0017] Found detection for foundGoBinary            
ERRO[0017] Found detection for foundGoBinary            
ERRO[0017] Found detection for foundGoBinary

In [22]:
# Task 3
pids = ['519423', '561559']

results = []
i = 0
for eachPid in pids: 
    command1 = 'sudo -S lsof -p {}'.format(pids[i])
    stdin, stdout, stderr = ssh.exec_command(command1)

    stdin.write(password + "\n")
    stdin.flush()

    results.append(stdout.readlines())
    i += 1 

i = 0 
for eachResult in results:
    print("*** Results for PID {} ***\n\n".format(pids[i]))
    for line in eachResult:
        print(line,end="")
    i += 1 
    print('\n')


*** Results for PID 519423 ***


COMMAND    PID USER   FD      TYPE             DEVICE SIZE/OFF    NODE NAME
snapd   519423 root  cwd       DIR                8,1     4096       2 /
snapd   519423 root  rtd       DIR                8,1     4096       2 /
snapd   519423 root  txt       REG                7,0 32457312     151 /snap/snapd/17336/usr/lib/snapd/snapd
snapd   519423 root  mem       REG                8,1   101320   21165 /usr/lib/x86_64-linux-gnu/libresolv-2.31.so
snapd   519423 root  mem       REG                8,1    27080   19483 /usr/lib/x86_64-linux-gnu/libnss_dns-2.31.so
snapd   519423 root  mem       REG                8,1    51832   19616 /usr/lib/x86_64-linux-gnu/libnss_files-2.31.so
snapd   519423 root  mem       REG                8,1  2029560    2575 /usr/lib/x86_64-linux-gnu/libc-2.31.so
snapd   519423 root  mem       REG                8,1   157224   20034 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
snapd   519423 root  mem       REG                8,1   19147

In [23]:
# Task 4:

print("*** Downloading Suspicious Files ***")

sftp = ssh.open_sftp()
sftp.get("/home/p.dalton/.d.log", "./Suspicious Files/.d.log")
sftp.get("/home/p.dalton/aqwerpuqwerj", "./Suspicious Files/aqwerpuqwerj.txt")
sftp.get("/home/p.dalton/test-lk/d.pid", "./Suspicious Files/d.pid")

print("*** Successfully downloaded \"/.d.log\", \"/aqwerpuqwerj\", and \"/d.pid\"")

*** Downloading Suspicious Files ***
*** Successfully downloaded "/.d.log", "/aqwerpuqwerj", and "/d.pid"


In [24]:
# Task 5

# I probably made this more complicated than it could've been, but I wanted to use lists to make my code shorter. My brain is built on controlled chaos, haha.

commands = [
"lsof -p 561559", 
"cat /etc/passwd | grep '^[a-z]\.[a-z]*'",
"cat /var/log/auth.log | grep p.dalton",
"last | grep p.dalton",
"ls /home/p.dalton/"
]

commandJob = [
"*** List of Processes on PID 561559 ***",
"*** Looking for suspicious account names ***",
"*** Searching Audit Logs ***",
"*** Last Login ***",
"*** Unauthorized Users Home Directory ***"
]

i = 0
for eachCommand in commands: 
    stdin,stdout,stderr = ssh.exec_command("sudo -S " + commands[i])
    stdin.write(password + "\n")
    stdin.flush()
    if i < 5:
        print("\n" + commandJob[i] + "\n")
    for line in iter(stdout.readline,""):
        
        print(line,end="")
    i += 1



*** List of Processes on PID 561559 ***

COMMAND    PID     USER   FD      TYPE  DEVICE SIZE/OFF    NODE NAME
egrep   561559 p.dalton  cwd       DIR     8,1     4096  258069 /home/p.dalton
egrep   561559 p.dalton  rtd       DIR     8,1     4096       2 /
egrep   561559 p.dalton  txt       REG     8,1  6597697  258326 /usr/local/bin/egrep
egrep   561559 p.dalton  mem       REG     8,1  2029560    2575 /usr/lib/x86_64-linux-gnu/libc-2.31.so
egrep   561559 p.dalton  mem       REG     8,1   157224   20034 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
egrep   561559 p.dalton  mem       REG     8,1   191472    2570 /usr/lib/x86_64-linux-gnu/ld-2.31.so
egrep   561559 p.dalton    0r      CHR     1,3      0t0       6 /dev/null
egrep   561559 p.dalton    1w      REG     8,1      850  258258 /home/p.dalton/.d.log
egrep   561559 p.dalton    2w      REG     8,1      850  258258 /home/p.dalton/.d.log
egrep   561559 p.dalton    3r      CHR     1,3      0t0       6 /dev/null
egrep   561559 p.dalton   